# GoogleGenerativeAIEmbeddings

这将帮助你使用LangChain开始使用Google Generative AI的[嵌入模型](/docs/concepts/embedding_models)。有关`GoogleGenerativeAIEmbeddings`功能和配置选项的详细文档，请参考[API参考文档](https://api.js.langchain.com/classes/langchain_google_genai.GoogleGenerativeAIEmbeddings.html)。

## 概述
### 集成详情

| 类 | 包 | 本地 | [Python支持](https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/) | 包下载量 | 最新包 |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [`GoogleGenerativeAIEmbeddings`](https://api.js.langchain.com/classes/langchain_google_genai.GoogleGenerativeAIEmbeddings.html) | [`@langchain/google-genai`](https://npmjs.com/@langchain/google-genai) | ❌ | ✅ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/google-genai?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/google-genai?style=flat-square&label=%20&) |

## 准备工作

要访问Google Generative AI的嵌入模型，你需要注册一个Google AI账户，获取一个API密钥，并安装`@langchain/google-genai`集成包。

### 凭据

在这里获取API密钥：https://ai.google.dev/tutorials/setup。

接下来，将你的密钥设置为名为`GOOGLE_API_KEY`的环境变量：

```bash
export GOOGLE_API_KEY="your-api-key"
```

如果你想获取模型调用的自动追踪，也可以通过取消注释以下内容来设置你的[LangSmith](https://docs.smith.langchain.com/) API密钥：

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### 安装

LangChain的`GoogleGenerativeAIEmbeddings`集成位于`@langchain/google-genai`包中。你也可以选择安装官方SDK：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/google-genai @langchain/core @google/generative-ai
</Npm2Yarn>
```

## 实例化

现在我们可以实例化我们的模型对象并嵌入文本：

In [1]:
import { GoogleGenerativeAIEmbeddings } from "@langchain/google-genai";
import { TaskType } from "@google/generative-ai";

const embeddings = new GoogleGenerativeAIEmbeddings({
  model: "text-embedding-004", // 768 dimensions
  taskType: TaskType.RETRIEVAL_DOCUMENT,
  title: "Document title",
});

## 索引与检索

嵌入模型通常用于检索增强生成（RAG）流程中，既用于数据索引，也用于后续的数据检索。有关更详细的说明，请参阅我们位于[使用外部知识教程](/docs/tutorials/#working-with-external-knowledge)下的RAG教程。

下方展示了如何使用我们之前初始化的`embeddings`对象进行数据索引和检索。在此示例中，我们将使用演示用的[`MemoryVectorStore`](/docs/integrations/vectorstores/memory)来索引和检索一个示例文档。

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## 直接使用

在底层，vectorstore 和 retriever 的实现分别通过调用 `embeddings.embedDocument(...)` 和 `embeddings.embedQuery(...)` 来为 `fromDocuments` 和 retriever 的 `invoke` 操作中使用的文本创建嵌入。

您可以直接调用这些方法，以用于您自己的使用场景。

### 嵌入单个文本

您可以使用 `embedQuery` 对查询进行嵌入，以用于搜索。这将生成针对该查询的向量表示：

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
  -0.018286658,   0.020051053,  -0.057487167,   0.0059406986, -0.0036901247,
  -0.010400916,    0.03396853,  -0.010867519,    0.015650319,   0.026443942,
   0.012251757,   -0.01581729,    0.02031182, -0.00062176475,  0.0065521155,
   -0.07107355,   0.033614952,    0.07109807,   -0.021078493,   0.048039366,
   0.022973344,    -0.0361746,   -0.04550704,   -0.048807852,    0.03414146,
   0.042450827,    0.02930612,   0.027274853,   -0.027707053,   -0.04167595,
    0.01708843,   0.028532283, -0.0018593844,      -0.096786,  -0.034648854,
  0.0013152987,   0.024425535,    0.04937838,    0.036890924,  -0.074619934,
  -0.028723065,   0.029158255,  -0.023993572,     0.03163398,   -0.02036324,
   -0.02333609,  -0.017407075, -0.0059643993,    -0.05564625,   0.051022638,
    0.03264913,  -0.008254581,  -0.030552095,    0.072952054,   -0.05448913,
   0.012030814,   -0.07978849,  -0.030417662,   0.0038343794,    0.03237516,
  -0.054259773,    -0.0524064,   -0.02145499,    0.006439614,    0.0498894

### 嵌入多个文本

您可以使用 `embedDocuments` 对多个文本进行索引嵌入。此方法的内部实现可能（但不一定）与嵌入查询的方式不同：

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
  -0.018286658,   0.020051053,  -0.057487167,   0.0059406986, -0.0036901247,
  -0.010400916,    0.03396853,  -0.010867519,    0.015650319,   0.026443942,
   0.012251757,   -0.01581729,    0.02031182, -0.00062176475,  0.0065521155,
   -0.07107355,   0.033614952,    0.07109807,   -0.021078493,   0.048039366,
   0.022973344,    -0.0361746,   -0.04550704,   -0.048807852,    0.03414146,
   0.042450827,    0.02930612,   0.027274853,   -0.027707053,   -0.04167595,
    0.01708843,   0.028532283, -0.0018593844,      -0.096786,  -0.034648854,
  0.0013152987,   0.024425535,    0.04937838,    0.036890924,  -0.074619934,
  -0.028723065,   0.029158255,  -0.023993572,     0.03163398,   -0.02036324,
   -0.02333609,  -0.017407075, -0.0059643993,    -0.05564625,   0.051022638,
    0.03264913,  -0.008254581,  -0.030552095,    0.072952054,   -0.05448913,
   0.012030814,   -0.07978849,  -0.030417662,   0.0038343794,    0.03237516,
  -0.054259773,    -0.0524064,   -0.02145499,    0.006439614,    0.0498894

## API 参考文档

如需详细了解所有 `GoogleGenerativeAIEmbeddings` 的功能和配置，请前往 API 参考文档：https://api.js.langchain.com/classes/langchain_google_genai.GoogleGenerativeAIEmbeddings.html